In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Reshape, concatenate, Conv1D, MaxPooling1D, Bidirectional, GRU, SimpleRNN, LSTM
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.optimizers import Adam
from keras.losses import mean_squared_error, mean_absolute_error

# load data and scale
#data = pd.read_csv('hourly_data.csv', parse_dates=['datetime'], index_col='datetime')
df=pd.read_csv('PRSA_data_2010.1.1-2014.12.31.csv')
df=df.iloc[:,1:-7]
df["Date"] = pd.to_datetime(df["year"].apply(str) + "/" + df["month"].apply(str) + "/" + df["day"].apply(str)+ " " + df["hour"].apply(str)+ ":" +"00")
df=df.drop(['year','month','day','hour'],axis=1)
df=df.dropna()
df=df.rename(columns={'Date':'DATE','pm2.5':'PM'})
data=df.set_index('DATE')
print(data)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# define input shape and number of time steps
n_steps = 24
n_features = 1
n_obs = n_steps*n_features

# split into train and test sets
train_size = int(len(data) * 0.8)
train, test = scaled_data[:train_size,:], scaled_data[train_size:,:]

# create time series data for training
X_train, y_train = [], []
for i in range(n_obs, len(train)):
    X_train.append(train[i-n_obs:i, 0])
    y_train.append(train[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = X_train.reshape((X_train.shape[0], n_steps, n_features))

# create time series data for testing
X_test, y_test = [], []
for i in range(n_obs, len(test)):
    X_test.append(test[i-n_obs:i, 0])
    y_test.append(test[i, 0])
X_test, y_test = np.array(X_test), np.array(y_test)
X_test = X_test.reshape((X_test.shape[0], n_steps, n_features))

# define model architecture
input_layer = Input(shape=(n_steps, n_features))
conv_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(input_layer)
pooling_layer = MaxPooling1D(pool_size=2)(conv_layer)
flatten_layer = Flatten()(pooling_layer)
reshape_layer = Reshape((n_steps, -1))(flatten_layer)
biLSTM_layer = Bidirectional(LSTM(units=50, return_sequences=True))(reshape_layer)
GRU_layer = GRU(units=50, return_sequences=True)(biLSTM_layer)
RNN_layer = SimpleRNN(units=50, return_sequences=True)(GRU_layer)
merge_layer = concatenate([biLSTM_layer, GRU_layer, RNN_layer], axis=-1)
flatten2_layer = Flatten()(merge_layer)
dropout_layer = Dropout(0.2)(flatten2_layer)
output_layer = Dense(units=1, activation='linear')(dropout_layer)
model = Model(inputs=input_layer, outputs=output_layer)

# define early stopping and compile model
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='mean_squared_error')

# train model and evaluate on test set
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stop])
test_loss = model.evaluate(X_test, y_test)

# predict on test set and inverse transform



2023-04-11 21:56:31.843897: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-11 21:56:31.888302: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-11 21:56:31.889305: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 21:56:32.988125: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


                        PM
DATE                      
2010-01-02 00:00:00  129.0
2010-01-02 01:00:00  148.0
2010-01-02 02:00:00  159.0
2010-01-02 03:00:00  181.0
2010-01-02 04:00:00  138.0
...                    ...
2014-12-31 19:00:00    8.0
2014-12-31 20:00:00   10.0
2014-12-31 21:00:00   10.0
2014-12-31 22:00:00    8.0
2014-12-31 23:00:00   12.0

[41757 rows x 1 columns]


ValueError: Exception encountered when calling layer "reshape" (type Reshape).

total size of new array must be unchanged, input_shape = [704], output_shape = [24, -1]

Call arguments received by layer "reshape" (type Reshape):
  • inputs=tf.Tensor(shape=(None, 704), dtype=float32)